In [1]:
import os
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from dtypes import dtypes

Project phases:
1) problem statement
2) solution design
3) solution development
4) data collection
5) writing

You can find 5 datasets :

- games.csv : all games from 2004 season to last update with the date, teams and some details like number of points, etc.
- games_details.csv : details of games dataset, all statistics of players for a given game
- players.csv : players details (name)
- ranking.csv : ranking of NBA given a day (split into west and east on CONFERENCE column
- teams.csv : all teams of NBA

***Problem statement:***

1) Predicting the most impactful player of a match

--> define the impact of a player
--> predictors?
--> label? (which variable to use for the impact - derived from its definition)

2) Predicting some statistics of a player in a match

--> define the variables (or single statistic variable derived from the set) that compose the player's statistics
--> predictors?

***Solution Design:***

1) 
- For the first problem we need to look at the players and matches, so we need to merge in some way the games_details.csv (that contains the statistics of the players for a given game) with players.csv (for single player's ID) and games.csv. Probably there is also a connection with ranking.csv, thinking for example of the motivation a player may have when his team is at the top or at the bottom of the rank. 
- ***Impactness definition***: number of scores? number of assists? Unifying statistics like that? The main question is: would the game be the same without that player? I think this is the main concept of impactness

2) For the second problem we have first to decide how to define a player's statistic/s

***Solution Development***

***Data Exploration***

In [5]:
games_details = pd.read_csv('data/games_details.csv', dtype = dtypes['games_details'])
games = pd.read_csv('data/games.csv', dtype = dtypes['games'])
players = pd.read_csv('data/players.csv', dtype = dtypes['players'])
ranking = pd.read_csv('data/ranking.csv', dtype = dtypes['rankings'])
teams = pd.read_csv('data/teams.csv', dtype = dtypes['teams'])

Problems right now:
- different player_id btw datasets --> leading to missing data
- different game_id btw datasets --> leading to missing data (maybe games registered in games.csv but not registered in games_details.csv?)

In [86]:
games_details['PLAYER_ID'].unique().shape

(2687,)

In [87]:
players['PLAYER_ID'].unique().shape

(1769,)

In [88]:
games['GAME_ID'].unique().shape 

(26622,)

In [89]:
games_details['GAME_ID'].unique().shape

(26523,)

In [90]:
# checking games that are in games but not in games_details
games_not_in_games_details = games[~games['GAME_ID'].isin(games_details['GAME_ID'])]

In [91]:
games_not_in_games_details['GAME_DATE_EST'].max(), games_not_in_games_details['GAME_DATE_EST'].min()

('2003-10-24', '2003-10-07')

From here we can see that the games not present in games.csv are in 2003 and that year should not be considered since it shouldn't be in the dataset. Solution: remove rows with year 2003 and select the game_id of that dataset. Subset the games_details.csv based on those IDs.

In [92]:
# Convert the GAME_DATE_EST column to datetime format
games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'])

# Extract the unique years
unique_years = games['GAME_DATE_EST'].dt.year.unique()
print(unique_years)

[2022 2021 2020 2019 2014 2013 2012 2011 2010 2009 2008 2007 2006 2005
 2004 2003 2018 2017 2016 2015]


In [93]:
# Filter out rows that include the year 2003
games_filtered = games[games['GAME_DATE_EST'].dt.year != 2003]

# Order the remaining rows based on the date
games_filtered = games_filtered.sort_values(by='GAME_DATE_EST')

# DataFrame
games_filtered = pd.DataFrame(games_filtered)
games_filtered.reset_index(drop=True, inplace=True)

games_filtered.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2004-01-02,20300451,Final,1610612764,1610612744,2003,1610612764,97.0,0.446,0.611,...,30.0,58.0,1610612744,79.0,0.403,0.818,0.381,20.0,36.0,1
1,2004-01-02,20300458,Final,1610612750,1610612737,2003,1610612750,93.0,0.409,0.929,...,32.0,56.0,1610612737,75.0,0.372,0.737,0.375,22.0,31.0,1
2,2004-01-02,20300454,Final,1610612752,1610612741,2003,1610612752,99.0,0.470,0.800,...,25.0,38.0,1610612741,104.0,0.488,0.724,0.385,20.0,44.0,0
3,2004-01-02,20300457,Final,1610612765,1610612756,2003,1610612765,93.0,0.389,0.947,...,26.0,54.0,1610612756,81.0,0.395,0.895,0.364,20.0,34.0,1
4,2004-01-02,20300453,Final,1610612738,1610612754,2003,1610612738,90.0,0.507,0.640,...,17.0,34.0,1610612754,103.0,0.437,0.760,0.333,21.0,47.0,0


In [94]:
games_filtered['GAME_ID'].unique().shape, games_details['GAME_ID'].unique().shape 

## STILL NEED TO CHECK WHY THE GAMES_DETAILS CONTAIN MORE GAMES THAN GAMES_FILTERED

## A POSSIBLE SOLUTION IS TO FILTER OUT THE GAMES THAT ARE NOT IN GAMES_DETAILS

((26058,), (26523,))

In [95]:
## FILTERING OUT THE GAMES THAT ARE NOT IN GAMES_DETAILS
games_details_filtered = games_details[games_details['GAME_ID'].isin(games_filtered['GAME_ID'])]
games_details_filtered['GAME_ID'].unique().shape, games_filtered['GAME_ID'].unique().shape

((26058,), (26058,))

Now games_filtered and games_details_filtered have the same number of unique game_ids
Need to do the same for the student_id

In [96]:
players['PLAYER_ID'].unique().shape, games_details['PLAYER_ID'].unique().shape, games_details_filtered['PLAYER_ID'].unique().shape

((1769,), (2687,), (2657,))

Possible motivation: maybe substitutes of the players?
Anyway, maybe there is no need to filter the number of player_IDs since I need to consider only one player at a time and the entire learning process is done on them.

In [97]:
players['SEASON'].unique()

array([2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009])

Now I understood that I need to filter the years since I only have player data for years in 2009-2019

In [98]:
players['PLAYER_ID'].unique().shape, players.shape

((1769,), (7228, 4))

In [103]:
## unifiying games_fltered and games_details_filtered
data_filtered = pd.merge(games_filtered, games_details_filtered, on='GAME_ID', how='inner')


In [104]:
## FILTERING YEARS OF GAMES_DETAILS_FILTERED AND GAMES_FILTERED TO MATCH THE PLAYERS DATAFRAME
# Extract the unique seasons from the players dataframe
unique_seasons = players['SEASON'].unique()

# Filter out rows that include the seasons that are not in the players dataframe
data_filtered_date = data_filtered[data_filtered['SEASON'].isin(unique_seasons)]

In [105]:
data_filtered_date['PLAYER_ID'].unique().shape, players['PLAYER_ID'].unique().shape

((1775,), (1769,))

Got only 6 players unknown now

In [ ]:
## checking players that are on the data_filtered_date but not in the players dataframe
players_not_in_data_filtered = data_filtered_date[~data_filtered_date['PLAYER_ID'].isin(players['PLAYER_ID'])]

554770    1628987
554808    1628987
555282    1628987
556675    1628778
557170    1628778
           ...   
571384    1628778
571729    1628778
571807    1628778
571961    1629216
572236    1628778
Name: PLAYER_ID, Length: 90, dtype: int64

In [109]:
players_not_in_data_filtered['PLAYER_ID'].unique()

array([1628987, 1628778, 1629760, 1629216, 1626218, 1629007])

In [121]:
data_filtered_date[data_filtered_date['PLAYER_ID'] == players_not_in_data_filtered['PLAYER_ID'].unique()[5]]

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
567716,2020-07-24,11900109,Final,1610612755,1610612763,2019,1610612755,90.0,0.432,0.833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
568009,2020-07-26,11900122,Final,1610612763,1610612745,2019,1610612763,104.0,0.386,0.778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
568280,2020-07-28,11900128,Final,1610612748,1610612763,2019,1610612748,110.0,0.429,0.719,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since these 6 players played mostly (exept 3 games of one player so useless in any case) later than 2019, I can remove them 

In [130]:
## removing the players with player_id players_not_in_data_filtered['PLAYER_ID'].unique() 
data_filtered_date = data_filtered_date[~data_filtered_date['PLAYER_ID'].isin(players_not_in_data_filtered['PLAYER_ID'].unique())]

In [131]:
data_filtered_date['PLAYER_ID'].unique().shape, players['PLAYER_ID'].unique().shape

((1769,), (1769,))

Ok now we have the same number of player_IDs and game_IDs, let's tidy things up

In [132]:
game_data = data_filtered_date 
players_data = players

In [136]:
if game_data['GAME_ID'].unique().shape == data_filtered_date['GAME_ID'].unique().shape:
    print("Game_IDs OK")

Game_IDs OK


In [135]:
if game_data['PLAYER_ID'].unique().shape == players_data['PLAYER_ID'].unique().shape:
    print("Players_IDs OK")

Players_IDs OK


Now the problem is that I have the game_data dataset that contains NAs, let's fix it

In [ ]:
game_data.isna().sum()[game_data.isna().sum() > 0] ## this are the columns with missing values

NICKNAME          383868
START_POSITION    238168
COMMENT           319792
MIN                64077
FGM                64077
FGA                64077
FG_PCT             64077
FG3M               64077
FG3A               64077
FG3_PCT            64077
FTM                64077
FTA                64077
FT_PCT             64077
OREB               64077
DREB               64077
REB                64077
AST                64077
STL                64077
BLK                64077
TO                 64077
PF                 64077
PTS                64077
PLUS_MINUS         75557
dtype: int64

I first need to determine the label and the predictor variables